In [1]:
import os, glob
from astropy.io import fits as pyfits
import numpy as np
import scipy.interpolate

# METHOD: BIN DOWN IMAGES

In [2]:
#http://scipy.github.io/old-wiki/pages/Cookbook/Rebinning

def congrid(a, newdims, method='linear', centre=False, minusone=False):
    '''Arbitrary resampling of source array to new dimension sizes.
    Currently only supports maintaining the same number of dimensions.
    To use 1-D arrays, first promote them to shape (x,1).
    
    Uses the same parameters and creates the same co-ordinate lookup points
    as IDL''s congrid routine, which apparently originally came from a VAX/VMS
    routine of the same name.

    method:
    neighbour - closest value from original data
    nearest and linear - uses n x 1-D interpolations using
                         scipy.interpolate.interp1d
    (see Numerical Recipes for validity of use of n 1-D interpolations)
    spline - uses ndimage.map_coordinates

    centre:
    True - interpolation points are at the centres of the bins
    False - points are at the front edge of the bin

    minusone:
    For example- inarray.shape = (i,j) & new dimensions = (x,y)
    False - inarray is resampled by factors of (i/x) * (j/y)
    True - inarray is resampled by(i-1)/(x-1) * (j-1)/(y-1)
    This prevents extrapolation one element beyond bounds of input array.
    '''
    if not a.dtype in [np.float64, np.float32]:
        a = np.cast[float](a)
    
    m1 = np.cast[int](minusone)
    ofs = np.cast[int](centre) * 0.5
    old = np.array( a.shape )
    ndims = len( a.shape )
    if len( newdims ) != ndims:
        print "[congrid] dimensions error. " \
              "This routine currently only support " \
              "rebinning to the same number of dimensions."
        return None
    newdims = np.asarray( newdims, dtype=float )    
    dimlist = []

    if method == 'neighbour':
        for i in range( ndims ):
            base = np.indices(newdims)[i]
            dimlist.append( (old[i] - m1) / (newdims[i] - m1) \
                            * (base + ofs) - ofs )
        cd = np.array( dimlist ).round().astype(int)
        newa = a[list( cd )]
        return newa
    
    elif method in ['nearest','linear']:
        # calculate new dims
        for i in range( ndims ):
            base = np.arange( newdims[i] )
            dimlist.append( (old[i] - m1) / (newdims[i] - m1) \
                            * (base + ofs) - ofs )
        # specify old dims
        olddims = [np.arange(i, dtype = np.float) for i in list( a.shape )]

        # first interpolation - for ndims = any
        mint = scipy.interpolate.interp1d( olddims[-1], a, kind=method )
        newa = mint( dimlist[-1] )

        trorder = [ndims - 1] + range( ndims - 1 )
        for i in range( ndims - 2, -1, -1 ):
            newa = newa.transpose( trorder )

            mint = scipy.interpolate.interp1d( olddims[i], newa, kind=method )
            newa = mint( dimlist[i] )

        if ndims > 1:
            # need one more transpose to return to original dimensions
            newa = newa.transpose( trorder )

        return newa
    elif method in ['spline']:
        oslices = [ slice(0,j) for j in old ]
        oldcoords = np.ogrid[oslices]
        nslices = [ slice(0,j) for j in list(newdims) ]
        newcoords = np.mgrid[nslices]

        newcoords_dims = range(np.rank(newcoords))
        #make first index last
        newcoords_dims.append(newcoords_dims.pop(0))
        newcoords_tr = newcoords.transpose(newcoords_dims)
        # makes a view that affects newcoords

        newcoords_tr += ofs        

        deltas = (np.asarray(old) - m1) / (newdims - m1)
        newcoords_tr *= deltas

        newcoords_tr -= ofs

        newa = scipy.ndimage.map_coordinates(a, newcoords)
        return newa
    else:
        print "Congrid error: Unrecognized interpolation type.\n", \
              "Currently only \'neighbour\', \'nearest\',\'linear\',", \
              "and \'spline\' are supported."
        return None  

# LOOP OVER FILES, SHRINK THEM, AND MODIFY HEADER ACCORDINGLY

In [4]:
# INPUT NEEDED: 
maindir = "/Users/lajoie/WCS_Test/"
if maindir[-1]!='/': maindir+='/'
outfile = 'congrid' # prefix on output files
bindim =  200       # downsample image size
imgsize= 2048       # individual input image size


binfactor = float(imgsize) / float(bindim)

os.chdir(maindir)
folderfiles = glob.glob("*.fits")

ic = 0
for ifile in folderfiles:
    hdu = pyfits.open(ifile)
    outhdu = pyfits.HDUList()

    # MODIFY EACH EXTENSION ACCORDINGLY:
    for iext in xrange(len(hdu)):
        hdr = hdu[iext].header
        data= hdu[iext].data
        
        hdr["CRPIX1"] = bindim/2
        hdr["CRPIX2"] = bindim/2
        hdr["CDELT1"] = hdr["CDELT1"]*binfactor
        hdr["CDELT2"] = hdr["CDELT2"]*binfactor
        hdr["CD1_1"]  = hdr["CD1_1"] *binfactor
        hdr["CD1_2"]  = hdr["CD1_2"] *binfactor
        hdr["CD2_1"]  = hdr["CD2_1"] *binfactor
        hdr["CD2_2"]  = hdr["CD2_2"] *binfactor
        
        if data is not None:
            data = congrid(data, [bindim,bindim])
            if hdr["EXTNAME"]: extname = hdr["EXTNAME"] 
            outhdu.append(pyfits.ImageHDU(data, header=hdr, name=extname))   
        else: outhdu.append(pyfits.ImageHDU(header=hdr))
                
    filename = outfile+ifile.split('.')[0]+".fits"
    outhdu.writeto(filename, clobber=True)  
    
    hdu.close()
    outhdu.close()
    ic += 1

# CREATE OPERATION FILE FOR QUIP

In [10]:
print '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>'
print '<QUIP_OPERATION_FILE xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" creator="WSS Executive" time="16:22:40.093Z" date="2017-06-14Z" version="6.0.1" operational="false" xsi:noNamespaceSchemaLocation="/Users/lajoie/TEL/WSS-6.0.1/Software/schema/quip_operation_file.xsd">'
print '    <CORRECTION_ID>R2017061401</CORRECTION_ID>'
print '    <OPERATION_TYPE>THUMBNAIL</OPERATION_TYPE>'

for i in xrange(ic):
    print "       <IMAGE_PATH>{:s}{:s}{:02d}.fits</IMAGE_PATH>".format(maindir,outfile,i)
    
print '       </IMAGES>'    
print '       <OUTPUT>'
print '           <OUTPUT_DIRECTORY>{:s}quip/</OUTPUT_DIRECTORY>'.format(maindir)
print '           <LOG_FILE_PATH>{:s}quip/R2017061401_quip_activity_log.xml</LOG_FILE_PATH>'.format(maindir)
print '           <OUT_FILE_PATH>{:s}quip/R2017061401_quip_out.xml</OUT_FILE_PATH>'.format(maindir)
print '       </OUTPUT>'

print '</QUIP_OPERATION_FILE>'

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<QUIP_OPERATION_FILE xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" creator="WSS Executive" time="16:22:40.093Z" date="2017-06-14Z" version="6.0.1" operational="false" xsi:noNamespaceSchemaLocation="/Users/lajoie/TEL/WSS-6.0.1/Software/schema/quip_operation_file.xsd">
    <CORRECTION_ID>R2017061401</CORRECTION_ID>
    <OPERATION_TYPE>THUMBNAIL</OPERATION_TYPE>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid00.fits</IMAGE_PATH>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid01.fits</IMAGE_PATH>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid02.fits</IMAGE_PATH>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid03.fits</IMAGE_PATH>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid04.fits</IMAGE_PATH>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid05.fits</IMAGE_PATH>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid06.fits</IMAGE_PATH>
       <IMAGE_PATH>/Users/lajoie/WCS_Test/congrid07.fits</IMAGE_PATH>
       <IMAGE_